# Task 2b: Attribute Detection (Viewing Angle) with EfficientDet

**Student:** Luigi Camilleri  
**Model:** EfficientDet (D0-D7, EfficientNet backbone) 

This notebook implements a detector for classifying the viewing angle of traffic signs using EfficientDet. It covers:
1.  **Dataset Preparation**: Loading images and annotations.
2.  **Model Training**: Fine Tuning a pretrained `EfficientDet` to detect and classify viewing angles (e.g., 'Frontal', 'Side', 'Oblique').
3.  **Evaluation**: Calculating F1-Scores.
4.  **Inference & Analytics**: Visualisation and counting detections of viewing angles.

In [ ]:
# --- Environment & Imports ---
from __future__ import annotations
import importlib, subprocess, sys, json, os

def ensure_package(pkg: str, import_name: str | None = None, pip_name: str | None = None):
    try:
        return importlib.import_module(import_name or pkg)
    except ImportError:
        pip_target = pip_name or pkg
        print(f"Installing missing package: {pip_target}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_target], stdout=subprocess.DEVNULL)
        return importlib.import_module(import_name or pkg)

torch = ensure_package("torch")
plt = ensure_package("matplotlib.pyplot", "matplotlib.pyplot", "matplotlib")
np = ensure_package("numpy")
PIL = ensure_package("Pillow", "PIL", "pillow")
time = ensure_package("time")
# EfficientDet: use efficientdet-pytorch or equivalent
try:
    efficientdet = ensure_package("efficientdet")
except Exception:
    print("Please install efficientdet-pytorch or provide the correct import path.")

from torch.utils.data import DataLoader
import torchvision.transforms as T
from PIL import Image
import time

torch.manual_seed(42)
print("Environment ready: all dependencies installed and imported.")

## 1. Dataset Preparation

I use the same `SignsDataset` class as before, but for viewing angle classification.

**Note:** Images are stored in `images` subfolder and annotations in `annotations` subfolder. These two subfolders are found in the `COCO-based_COCO_viewing_angle` folder.

In each subfolder there are the data split into `train`, `val` and `test`. This is done to make sure that the results are reproducible.

In [ ]:
DATA_DIR_TRAIN = "Assignment Material/COCO-based_COCO_viewing_angle/images/train" 
ANNOTATION_FILE_TRAIN = "Assignment Material/COCO-based_COCO_viewing_angle/annotations/train.json"

DATA_DIR_VAL = "Assignment Material/COCO-based_COCO_viewing_angle/images/val"
ANNOTATION_FILE_VAL = "Assignment Material/COCO-based_COCO_viewing_angle/annotations/val.json"

In [ ]:
# Build mapping: class_id -> class_name
with open(ANNOTATION_FILE_TRAIN, "r") as f:
    data = json.load(f)
CLASS_ID_TO_NAME = {cat["id"]: cat["name"] for cat in data["categories"]}
NUM_CLASSES = len(CLASS_ID_TO_NAME)

### Initialize Dataset and DataLoaders

In [ ]:
transform = T.Compose([T.ToTensor()])

# You may need to adapt this to your EfficientDet dataset class
train_dataset = SignsDataset(
    root=DATA_DIR_TRAIN,
    annFile=ANNOTATION_FILE_TRAIN,
    transforms=transform
)

val_dataset = SignsDataset(
    root=DATA_DIR_VAL,
    annFile=ANNOTATION_FILE_VAL,
    transforms=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda x: tuple(zip(*x)))

val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=lambda x: tuple(zip(*x)))

## 2. Model Configuration
Loading the EfficientDet model pre-trained on COCO and adapting proper number of classes.

In [ ]:
# Example: Load EfficientDet model (replace with actual EfficientDet loading code)
from efficientdet import EfficientDet # Replace with correct import if needed
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = EfficientDet(num_classes=NUM_CLASSES, compound_coef=0) # D0 variant, adjust as needed
model.to(device)

lr = 0.0025
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## 3. Training Loop
Training for 20 epochs and evaluating on validation set after each epoch.

In [ ]:
num_epochs = 20
train_losses = []
val_f1_scores = []
best_model = None
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # Training step (replace with EfficientDet-specific training code)
    train_loss = model.train_on_loader(train_loader, optimizer) # Placeholder
    train_losses.append(train_loss)

    # Validation step (replace with EfficientDet-specific validation code)
    val_f1 = model.evaluate_on_loader(val_loader) # Placeholder
    val_f1_scores.append(val_f1)

    if train_loss < best_loss:
        best_loss = train_loss
        best_model = model

    print(f"Loss: {train_loss:.4f} | F1: {val_f1:.4f}")

### Plotting the losses and F1-score for the viewing angle

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
plt.title("Training Losses Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), val_f1_scores, label='Validation F1-score', marker='o')
plt.title("Validation F1-score vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("F1-score")
plt.legend()
plt.grid(True)
plt.show()

## 4. Analytics & Visualization

Analyze the detections on the validation set. Count the number of detected viewing angles and visualize the results. Apply inference on the validation set which are unlabelled unseen examples.

In [ ]:
# Inference and summary (replace with EfficientDet-specific code)
model.eval()
total_detected_angles = 0
results_summary = []
with torch.no_grad():
    for img, target in val_dataset:
        prediction = model(img.unsqueeze(0).to(device)) # Placeholder
        # Postprocess prediction as needed for EfficientDet
        # Example: keep = prediction['scores'] > threshold
        # final_labels = prediction['labels'][keep]
        count = 0 # Replace with actual count
        total_detected_angles += count
        results_summary.append({
            "ImageID": target["image_id"].item(),
            "DetectedAngles": count
        })
print(f"Total viewing angles detected in validation set: {total_detected_angles}")
for res in results_summary:
    print(res)

### Save the model

In [ ]:
import os
os.makedirs("./models", exist_ok=True)
# Save best model (replace with EfficientDet-specific saving code)
torch.save(model.state_dict(), "./models/efficientdet_best.pt") # Placeholder

### Qualitative results and Visualisation

In [ ]:
# Visualize predictions (replace with EfficientDet-specific code)
for i in range(len(val_dataset)):
    img, target = val_dataset[i]
    with torch.no_grad():
        prediction = model(img.unsqueeze(0).to(device)) # Placeholder
    # visualize_predictions(img, prediction, CLASS_ID_TO_NAME) # Implement as needed

## Clean up memory

In [ ]:
import gc
import torch
import matplotlib.pyplot as plt

plt.close("all")
torch.cuda.empty_cache()
gc.collect()